In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, json
from urllib import request, parse
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
from time import sleep
import time

In [2]:
#json 데이터 불러오기
with open('Gameinfo_0_to_48336_parsed.json') as f:
    match_df = json.load(f)
#챔피언 id/이름 목록 파일 불러오기
with open('champion_id_and_name_list.json') as f1:
    champ_id_dict=json.load(f1)

In [3]:
champ_id_dict

{'266': 'Aatrox',
 '103': 'Ahri',
 '84': 'Akali',
 '12': 'Alistar',
 '32': 'Amumu',
 '34': 'Anivia',
 '1': 'Annie',
 '523': 'Aphelios',
 '22': 'Ashe',
 '136': 'Aurelion Sol',
 '268': 'Azir',
 '432': 'Bard',
 '53': 'Blitzcrank',
 '63': 'Brand',
 '201': 'Braum',
 '51': 'Caitlyn',
 '164': 'Camille',
 '69': 'Cassiopeia',
 '31': "Cho'Gath",
 '42': 'Corki',
 '122': 'Darius',
 '131': 'Diana',
 '119': 'Draven',
 '36': 'Dr. Mundo',
 '245': 'Ekko',
 '60': 'Elise',
 '28': 'Evelynn',
 '81': 'Ezreal',
 '9': 'Fiddlesticks',
 '114': 'Fiora',
 '105': 'Fizz',
 '3': 'Galio',
 '41': 'Gangplank',
 '86': 'Garen',
 '150': 'Gnar',
 '79': 'Gragas',
 '104': 'Graves',
 '120': 'Hecarim',
 '74': 'Heimerdinger',
 '420': 'Illaoi',
 '39': 'Irelia',
 '427': 'Ivern',
 '40': 'Janna',
 '59': 'Jarvan IV',
 '24': 'Jax',
 '126': 'Jayce',
 '202': 'Jhin',
 '222': 'Jinx',
 '145': "Kai'Sa",
 '429': 'Kalista',
 '43': 'Karma',
 '30': 'Karthus',
 '38': 'Kassadin',
 '55': 'Katarina',
 '10': 'Kayle',
 '141': 'Kayn',
 '85': 'Kennen'

In [4]:
match_df.keys()

dict_keys(['gameId', 'teamBlue', 'teamRed', 'teamBlueParticipants', 'teamRedParticipants'])

In [5]:
match_df['teamBlueParticipants']

{'0': [['헤이트 중궈', 163],
  ['냥이눈사람', 80],
  ['glwy', 67],
  ['mid gap im sorry', 517],
  ['rap starrr', 2]],
 '1': [['CHERRY', 201],
  ['나 도일이', 81],
  ['Bellwest', 98],
  ['Alpaca boy', 246],
  ['jungle is weak', 76]],
 '2': [['다훈갓', 142],
  ['sd11a5sd', 202],
  ['상록중', 126],
  ['CHERRY', 412],
  ['나 도일이', 876]],
 '3': [['김병준수', 84],
  ['SibasakiRoka', 43],
  ['상록중', 57],
  ['CHERRY', 81],
  ['나 도일이', 64]],
 '4': [['mid gap im sorry', 517],
  ['leileide', 2],
  ['SibasakiRoka', 236],
  ['스푸닝 최선영', 201],
  ['상록중', 516]],
 '5': [['외쳐주유소갓라카', 16],
  ['Bvvoy', 81],
  ['종학잇', 61],
  ['rensheng wandan', 517],
  ['MsHg', 164]],
 '6': [['암어바나나나', 84],
  ['YOLIWW', 104],
  ['AF Na Kyung', 875],
  ['인간이아닌사람임', 236],
  ['상록중', 117]],
 '7': [['dingji', 59],
  ['cautious mid', 61],
  ['corgii', 58],
  ['업보청산1', 222],
  ['Thresh Only', 412]],
 '8': [['애린이의 기욤이', 80],
  ['Blank', 120],
  ['Humble SUP', 89],
  ['puluxiuka', 29],
  ['몰아쳤던 폭풍', 875]],
 '9': [['SANDBOX Muru', 201],
  ['down sleep', 876],

In [5]:
driver= webdriver.Chrome()

In [6]:
def parse_tup(summonerName, championId): #동원님 스크립트를 이용해서 함수 재조합.
    global driver, champ_id_dict
    name_parsed = parse.quote_plus(summonerName)
    champ = champ_id_dict[championId] #id -챔피언명 정보 딕셔너리로 불러와주세요. 딕셔너리 이름 다르게 했으면 이 이름으로 하셔야 합니다. 
    try:
        driver.get(f'https://www.op.gg/summoner/userName={name_parsed}')
        driver.find_element_by_xpath('//*[@id="right_gametype_soloranked"]').click()
        sleep(2)
        x=driver.find_element_by_xpath('//*[@id="GameAverageStatsBox-matches"]/div[1]/table/tbody/tr[2]/td[2]/div[2]')
        spans=x.find_elements_by_xpath('./span')
        recentKDA= float(spans[0].text.split(':')[0])
        recentW_R= int(spans[1].find_element_by_xpath('./span').text[:-1])
        totalW_R=int(driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[2]/div[1]/div[1]/div/div[2]/div[3]/span[2]/span[3]').text.split()[-1][:-1])
        driver.get(f'https://www.op.gg/summoner/champions/userName={name_parsed}') #url 변경으로 대체가능
        driver.find_element_by_xpath('//*[@id="champion_season"]/li[2]/a').click()
        sleep(2)
        chamW_R=0
        chamKDA=0
        matchCNT=0
        table = driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[3]/div/div/div[2]/div[1]/table/tbody')
        trs=table.find_elements_by_xpath('./tr')
        #픽 champion의 시즌 승률 검색 만약 데이터가 없으면 최근승률로 대체
        for tr in trs:
                tds = tr.find_elements_by_xpath('./td')
                if tds[2].get_attribute('data-value')==champ:
                    chamW_R=int(tds[3].find_element_by_xpath('.//span').text[:-1])
                    chamKDA=float(tds[4].get_attribute("data-value"))
                    divs=tds[3].find_elements_by_xpath('./div/div/div')
                    matchCNT=int(divs[1].text[:-1])+int(divs[3].text[:-1])
                    break
        else:
            chamW_R=recentW_R
            chamKDA=recentKDA
                
        
        return [summonerName,champ,chamW_R,chamKDA,recentW_R,recentKDA, totalW_R ,matchCNT]
    except: #추출 불가면 아무것도 return하지 않음.
        return 'err!'
    

In [7]:
data_list = []
data_list_failed =[]

In [8]:
url_f = 'https://www.op.gg/summoner/userName={}'
#언어 설정 영어로 지정

i = 0
for i in range(1):
#for i in range(0, range(len(match_df['gameId'])), 100):
    driver= webdriver.Chrome() #드라이버를 열어야 작동.
    driver.get('https://op.gg/')
    driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
    sleep(2)
    lang_list = driver.find_elements_by_class_name('setting-list__item')
    for lang in lang_list:
        if "English" in lang.text:
            lang.click()
    sleep(2)
    driver.find_elements_by_class_name('setting__button')[0].click()
    for j in range(10):
    #for j in tqdm(range(100)): # 100단위로 처리     
    #userlist parsing
        try: 
            team_blue = {'gameId':match_df['gameId'][str(i+j)], 'teamId':100 }
            team_red = {'gameId':match_df['gameId'][str(i+j)], 'teamId':200 }
            team_blue['win'] = True if match_df['teamBlue'][str(i+j)][1] in ['Win', True, 'True'] else False
            team_red['win'] = True if match_df['teamRed'][str(i+j)][1] in ['Win', True, 'True'] else False
            #우선 둘이 같으면 데이터 오류로 판단하고 기각
            dat_err = False
            if team_blue['win'] == team_red['win']:
                dat_err = True
            team_blue['participants'] = []
            team_red['participants'] = []
            temp_blue =[]
            temp_red = []
            if not dat_err:
                for k in [0,5]: #blue : k=0, red : k=5
                    for l in range(5): #participantIdentities 에서 userid 가져옴,
                        if k == 0: p_name = match_df['teamBlueParticipants'][str(i+j)][l][0]
                        else: p_name = match_df['teamRedParticipants'][str(i+j)][l][0]
                        name_parsed = parse.quote_plus(p_name)
                        url = url_f.format(name_parsed)
                        html = request.urlopen(url).read().decode('utf8')
                        soup = BeautifulSoup(html, 'html.parser')
                        title = soup.title.string
                        if title[-1] == 'G': #반복 횟수 줄이기 위한 테크닉
                            break
                        elif k ==0: 
                            p_champ = match_df['teamBlueParticipants'][str(i+j)][l][1]
                            tup = (p_name, p_champ)
                            temp_blue.append(tup)
                        elif k ==5: #페이지 유효
                            p_champ = match_df['teamRedParticipants'][str(i+j)][l][0]
                            tup = (p_name, p_champ)
                            temp_red.append(tup)


                if len(temp_blue)==5: #5명 모두 긁어올 때만
                    parsed_res =[]
                    for tup in temp_blue:
                        res = parse_tup(*tup)
                        parsed_res.append(res)
                    if not ('err!' in parsed_res): 
                        team_blue['participants'] = parsed_res
                        data_list.append(team_blue)
                        #print('Blue add')

                if len(temp_red)==5: #5명 모두 긁어올 때만.
                    parsed_res =[]
                    for tup in temp_red:
                        res = parse_tup(*tup)
                        parsed_res.append(res)
                    if not ('err!' in parsed_res): 
                        team_red['participants'] = parsed_res
                        data_list.append(team_red)
        except:
            data_list_failed.append(i+j)
        
    with open(f'Gameinfo_with_user_0_to_48336_{i//100+1}.json', 'w') as pk1: #100개 긁어올 때마다...
        json.dump(data_list, pk1)
        data_list = []
    
    driver.close() #100개 모을 때마다 강제휴식 
    sleep(10)
                                                  
        
            


In [30]:
data_list_failed

[]

In [ ]:
with open('Gameinfo_with_user_0_to_48336.pickle', 'wb') as pk1:
    pickle.dump(data_list, pk1)
with open('Gameinfo_with_user_0_to_48336.json', 'w') as pk2:
    json.dump(data_list, pk2)

In [1]:
driver

NameError: name 'driver' is not defined